In [1]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2023-08-03 17:30:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [           <=>      ]  41.00M  15.6MB/s    in 2.6s    

2023-08-03 17:30:58 (15.6 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [2]:
from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bab754a808822659/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
# Peek at the first few examples
#drug_sample[:3]

In [4]:
sample = []
i=0
for i in range(3):
    sample.append(drug_sample[i])
sample

[{'Unnamed: 0': 87571,
  'drugName': 'Naproxen',
  'condition': 'Gout, Acute',
  'review': '"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  'rating': 9.0,
  'date': 'September 2, 2015',
  'usefulCount': 36},
 {'Unnamed: 0': 178045,
  'drugName': 'Duloxetine',
  'condition': 'ibromyalgia',
  'review': '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."'

In [5]:
for split in drug_dataset.keys():
    if len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0")):
        print("there are unique numbers of patient IDs")
    else:
        print("patient IDs are not unique")

there are unique numbers of patient IDs
there are unique numbers of patient IDs


In [6]:
drug_dataset.keys()

dict_keys(['train', 'test'])

In [7]:
drug_dataset = drug_dataset.rename_column("Unnamed: 0","patient_id")
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [8]:
def filter_nones(x):
    return {"condition": x["condition"] is not None}

In [9]:
drug_dataset = drug_dataset.map(filter_nones)

  0%|          | 0/161297 [00:00<?, ?ex/s]

  0%|          | 0/53766 [00:00<?, ?ex/s]

In [10]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}

drug_dataset.map(lowercase_condition)

  0%|          | 0/161297 [00:00<?, ?ex/s]

  0%|          | 0/53766 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [11]:
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

  0%|          | 0/162 [00:00<?, ?ba/s]

  0%|          | 0/54 [00:00<?, ?ba/s]

In [12]:
def review_len(rows):
    return {"review length" : len(rows["review"].split())}

In [13]:
drug_dataset = drug_dataset.map(review_len)


  0%|          | 0/161297 [00:00<?, ?ex/s]

  0%|          | 0/53766 [00:00<?, ?ex/s]

In [14]:
drug_dataset["train"][0] 

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'true',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review length': 17}

In [15]:
#drug_dataset["review length"] = lambda x: len(x['review length'].split())

In [ ]:
#drug_dataset["train"].sort("review length", reverse = True)[:3]

In [17]:
drug_dataset.num_rows

{'train': 161297, 'test': 53766}

In [18]:
drug_dataset = drug_dataset.filter(lambda x: x["review length"] > 30)
print(drug_dataset.num_rows)

  0%|          | 0/162 [00:00<?, ?ba/s]

  0%|          | 0/54 [00:00<?, ?ba/s]

{'train': 139280, 'test': 46352}


In [19]:
import html
text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [20]:
%%time
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

  0%|          | 0/139280 [00:00<?, ?ex/s]

  0%|          | 0/46352 [00:00<?, ?ex/s]

CPU times: user 26.2 s, sys: 374 ms, total: 26.6 s
Wall time: 26.5 s


In [21]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review length'],
        num_rows: 139280
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review length'],
        num_rows: 46352
    })
})

In [22]:
%time
drug_dataset = drug_dataset.map(lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched = True)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs


  0%|          | 0/140 [00:00<?, ?ba/s]

  0%|          | 0/47 [00:00<?, ?ba/s]

In [23]:
%time 
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")




CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs


In [24]:
%%time
def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [25]:
%%time
tokenized_dataset = drug_dataset.map(tokenize_function)


  0%|          | 0/139280 [00:00<?, ?ex/s]

  0%|          | 0/46352 [00:00<?, ?ex/s]

CPU times: user 2min 8s, sys: 1.03 s, total: 2min 9s
Wall time: 2min 8s


In [26]:
%%time
tokenized_dataset = drug_dataset.map(tokenize_function, batched = True, num_proc = 8)

#0:   0%|          | 0/18 [00:00<?, ?ba/s]

#2:   0%|          | 0/18 [00:00<?, ?ba/s]

#1:   0%|          | 0/18 [00:00<?, ?ba/s]

#3:   0%|          | 0/18 [00:00<?, ?ba/s]

#4:   0%|          | 0/18 [00:00<?, ?ba/s]

...

#4:   0%|          | 0/6 [00:00<?, ?ba/s]

#5:   0%|          | 0/6 [00:00<?, ?ba/s]

#7:   0%|          | 0/6 [00:00<?, ?ba/s]

#6:   0%|          | 0/6 [00:00<?, ?ba/s]

CPU times: user 1.31 s, sys: 425 ms, total: 1.73 s
Wall time: 35.3 s


In [27]:
%%time
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)


def slow_tokenize_function(examples):
    return slow_tokenizer(examples["review"], truncation=True)


tokenized_dataset = drug_dataset.map(slow_tokenize_function, batched=True, num_proc=8)

#0:   0%|          | 0/18 [00:00<?, ?ba/s]

#1:   0%|          | 0/18 [00:00<?, ?ba/s]

#2:   0%|          | 0/18 [00:00<?, ?ba/s]

#4:   0%|          | 0/18 [00:00<?, ?ba/s]

#3:   0%|          | 0/18 [00:00<?, ?ba/s]

...

#5:   0%|          | 0/6 [00:00<?, ?ba/s]

#4:   0%|          | 0/6 [00:00<?, ?ba/s]

#6:   0%|          | 0/6 [00:00<?, ?ba/s]

#7:   0%|          | 0/6 [00:00<?, ?ba/s]

CPU times: user 1.37 s, sys: 413 ms, total: 1.78 s
Wall time: 2min 38s


In [28]:
%%time
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.3 µs


In [29]:
result = tokenize_and_split(drug_dataset["train"][0])
[len(inp) for inp in result["input_ids"]]

[128, 49]

In [32]:
#tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names
)

  0%|          | 0/140 [00:00<?, ?ba/s]

  0%|          | 0/47 [00:00<?, ?ba/s]

In [33]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result